# Extract Simons Observatory instrument parameters

In order to be independent of `sotodlib` and track versions of the instrument model parameters more easily, we extract them to a file included in this repository.

The versioning will be YYYY.MM

In [1]:
import numpy as np

In [2]:
import sotodlib.sim_hardware as hardware

hw = hardware.get_example()

In [3]:
import sotodlib

In [4]:
sotodlib.__version__

'0.4.0+519.g70028af'

In [5]:
fwhm_arcmin = {"SA" : hw.data["telescopes"]["SAT1"]["fwhm"],
"LA":hw.data["telescopes"]["LAT"]["fwhm"]}

In [6]:
from pathlib import Path

In [7]:
dataset_name = Path("simonsobs_instrument_parameters_2022.08")

In [8]:
!mkdir -p $dataset_name

In [9]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [10]:
channels = []
for band_label, band_properties in hw.data["bands"].items():
    telescope = band_label[:3]
    print(band_label)
    ch = OrderedDict()
    ch["telescope"] = telescope
    ch["band"] = band_label
    ch["fwhm"] = fwhm_arcmin[telescope[:2]][band_label] * u.arcmin
    properties = hw.data["bands"][band_label]
    for freq_type in ["center", "low", "high"]:
        ch[freq_type + "_frequency"] = band_properties[freq_type] *u.GHz
    channels.append(ch)

LAT_f030
LAT_f040
LAT_f090
LAT_f150
LAT_f230
LAT_f290
SAT_f030
SAT_f040
SAT_f090
SAT_f150
SAT_f230
SAT_f290


## Bandpasses

Revision `b065fa3854b9a0f36bc9957bb94ec1f9481a5f2a` of https://github.com/simonsobs/instrument_model/

In [11]:
ls ../../../instrument_model/instrument_hardware/modeled_bandpasses/UHF

UHF_wOMT_LAT_beam.txt  UHF_wOMT_SAT_beam.txt  UHF_wOMT.txt


In [12]:
bandpass_files = [
    "../../../instrument_model/instrument_hardware/modeled_bandpasses/LF/LF_skinnyfatter.txt",
    "../../../instrument_model/instrument_hardware/modeled_bandpasses/MF/MF_wOMT_wLPF_v1.txt",
    "../../../instrument_model/instrument_hardware/modeled_bandpasses/UHF/UHF_wOMT.txt",
]

In [13]:
from astropy.io import ascii

for f in bandpass_files:
    modeled_bandpasses = ascii.read(f)
    for band in modeled_bandpasses.keys()[1:]:
        freq = int(band.split()[0])
        print(freq)
        if freq == 220:
            freq = 230
        if freq == 280:
            freq = 290
        freq_label = "Freq" if freq < 50 else "Frequency (GHz)"
        QTable({
        "bandpass_frequency":modeled_bandpasses[freq_label]*u.GHz,
        "bandpass_weight":modeled_bandpasses[band]
        }).write(dataset_name / f"bandpass_f{freq:03d}.tbl", format="ascii.ipac", overwrite=True)

30
40
90
150
220
280


In [14]:
instrument_parameters = QTable(channels)

In [15]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')"]

In [16]:

instrument_parameters.meta["comments"] = ["sotodlib_version:" + sotodlib.__version__] + ipac_warning

In [17]:
instrument_parameters

telescope,band,fwhm,center_frequency,low_frequency,high_frequency
,,arcmin,GHz,GHz,GHz
str3,str8,float64,float64,float64,float64
LAT,LAT_f030,7.4,25.7,21.7,29.7
LAT,LAT_f040,5.1,38.9,30.9,46.9
LAT,LAT_f090,2.2,92.0,79.0,105.0
LAT,LAT_f150,1.4,147.5,130.0,165.0
LAT,LAT_f230,1.0,225.7,196.7,254.7
LAT,LAT_f290,0.9,285.4,258.4,312.4
SAT,SAT_f030,91.0,25.7,21.7,29.7
SAT,SAT_f040,63.0,38.9,30.9,46.9


In [18]:
instrument_parameters.write(dataset_name / (dataset_name.name+".tbl"), format="ascii.ipac", overwrite=True)